# Brain Tumor Classification using Deep Learning
## Using VGG16 for MRI Image Classification

### 1. Setup and Data Download
Setting up Kaggle credentials and downloading the brain tumor MRI dataset

In [1]:
from google.colab import files
files.upload()  # upload kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
 87% 129M/149M [00:00<00:00, 1.34GB/s]
100% 149M/149M [00:00<00:00, 1.27GB/s]


In [ ]:
!unzip brain-tumor-mri-dataset.zip

### 2. Import Libraries and Setup GPU

Importing required PyTorch libraries and setting up CUDA device

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [5]:
device = torch.device("cuda")

In [6]:
print(torch.cuda.get_device_name(0))

Tesla T4


### 3. Data Preprocessing

Defining data transformations for training and testing datasets:
- Random horizontal flips
- Random rotations 
- Color jittering
- Random resized crops
- Normalization

In [36]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),       # flip with 50% chance
    transforms.RandomRotation(15),                # rotate within ±15 degrees
    transforms.ColorJitter(brightness=0.2,
                           contrast=0.2,
                           saturation=0.2,
                           hue=0.1),             # random color changes
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)), # random crop & resize
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

### 4. Data Loading 

Creating datasets and dataloaders:
- Training dataset from '/content/Training'
- Testing dataset from '/content/Testing'
- Batch size: 32

In [37]:
train_dataset = datasets.ImageFolder(root='/content/Training',transform=train_transform)
test_dataset = datasets.ImageFolder(root='/content/Testing',transform=test_transform)

In [38]:
train_loader = DataLoader(dataset=train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=32)

### 5. Model Architecture

Loading and modifying VGG16 model:
- Using pretrained VGG16
- Freezing feature extraction layers
- Modifying classifier layers for 4 classes

In [39]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
for params in vgg16.features.parameters():
  params.requires_grad=False

In [41]:
import torch.nn as nn
import torch

In [42]:
vgg16.classifier=nn.Sequential(
    nn.Linear(25088,64),
    nn.ReLU(),
    nn.Linear(64,128),
    nn.ReLU(),
    nn.Linear(128,4)
)

### 6. Training Configuration

Setting up training parameters:
- Epochs: 10
- Learning rate: 0.01
- Loss function: CrossEntropyLoss 
- Optimizer: Adam

In [43]:
epochs = 10
lr = 0.01

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(),lr=lr)

In [45]:
vgg16.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### 6. Training Configuration

Setting up training parameters:
- Epochs: 10
- Learning rate: 0.01
- Loss function: CrossEntropyLoss 
- Optimizer: Adam

In [46]:
for epoch in range(epochs):
  total_loss=0
  for batch_features,batch_labels in train_loader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)
    optimizer.zero_grad()
    #forward pass
    out = vgg16(batch_features)
    #loss calculation
    loss = criterion(out,batch_labels)
    #backward pass
    loss.backward()
    #weight updates
    optimizer.step()
    total_loss+=loss.item()
  print(f"epoch:{epoch+1},loss:{total_loss/len(train_loader)}")

epoch:1,loss:0.7373745477649086
epoch:2,loss:0.2910242142915393
epoch:3,loss:0.21168455158948232
epoch:4,loss:0.1838727814546011
epoch:5,loss:0.19325490870289297
epoch:6,loss:0.17169637601905863
epoch:7,loss:0.17295994533891867
epoch:8,loss:0.14894290303280733
epoch:9,loss:0.1306944888058462
epoch:10,loss:0.12287652678137344


### 8. Evaluation

Model evaluation:
- Computing validation loss
- Computing accuracy metrics
- Testing on validation set

In [47]:
def evaluate(model, test_loader, criterion, device):
    model.eval()  # evaluation mode
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # disable gradients
        for batch_features, batch_labels in test_loader:
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_features)          # forward pass
            loss = criterion(outputs, batch_labels)  # compute loss
            total_loss += loss.item()

            # Predictions
            _, predicted = torch.max(outputs, 1)     # get class with max score
            correct += (predicted == batch_labels).sum().item()
            total += batch_labels.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [48]:
val_loss, val_acc = evaluate(vgg16, test_loader, criterion, device)
print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

Validation Loss: 0.2021, Accuracy: 94.20%


### 9. Save Model

Saving the trained model weights

In [49]:
torch.save(vgg16.state_dict(),"model.pth")